## Notes

"As I explained before: you have to do detection with py-faster-rcnn and then a tracking using deep sort."

## Use py-faster-rcnn to get detections in ROI format

# Main

In [ ]:
import generate_detections
from deep_sort.detection import Detection

def main():
    # Start up your application, then generate the encoder function.
    encoder = generate_detections.create_box_encoder(
        PATH_TO_DEEPSORT_TENSORFLOW_CHECKPOINT_FILE)

    # Now, enter the application main loop. Should be something like this:
    for image, detections in my_data_source:
        # - image is a BGR color image
        # - detections is an Nx4 matrix of bounding boxes in
        #   format (x, y, w, h) where (x, y) is the top-left corner
        #   and (w, h) is the extent
        # - features is an Nx128 matrix of corresponding appearance descriptors
        features = encoder(image, detections)
        
        # Generate a list of detections (we simply set the confidence
        # score to 1.0 here).
        detections = [
            Detection(bbox, 1.0, feature) for bbox, feature in
            zip(detections, features)]

        # Call the tracker
        tracker.predict()
        tracker.update(detections)

        # Go on and visualize the results, etc.

# Generate Detections

In [ ]:
def generate_detections_video(encoder, video, name_out, detection):

    """Generate detections with features. Modification with video
    """

    detections_in = detection
    detections_out = []

    frame_indices = detections_in[:, 0].astype(np.int)
    min_frame_idx = frame_indices.astype(np.int).min()
    max_frame_idx = frame_indices.astype(np.int).max()

    camera = cv2.VideoCapture(video)

    for frame_idx in range(min_frame_idx, max_frame_idx + 1):
        print("Frame %05d/%05d" % (frame_idx, max_frame_idx))
        mask = frame_indices == frame_idx
        rows = detections_in[mask]

        if frame_idx not in frame_indices:
            print("WARNING could not find image for frame %d" % frame_idx)
            continue
        camera.set(cv2.CAP_PROP_POS_FRAMES, frame_idx-1);
        (grabbed, bgr_image) = camera.read()
        # bgr_image = cv2.imread(image_filenames[frame_idx], cv2.IMREAD_COLOR)
        rows[:, 4:6] -= rows[:, 2:4]  # convert to [x1,y1,x2,y2] to [x1,y1,w,h]
        features = encoder(bgr_image, rows[:, 2:6].copy())
        detections_out += [np.r_[(row, feature)] for row, feature
                           in zip(rows, features)]

    # output_filename = os.path.join(output_dir, "%s.npy" % sequence)
    np.save(name_out, np.asarray(detections_out), allow_pickle=False)

In [ ]:
def generate_detections_video(encoder, video, name_out, detection):

    """Generate detections with features. Modification with video
    """

    detections_in = detection
    detections_out = []

    frame_indices = detections_in[:, 0].astype(np.int)
    min_frame_idx = frame_indices.astype(np.int).min()
    max_frame_idx = frame_indices.astype(np.int).max()

    camera = cv2.VideoCapture(video)

    for frame_idx in range(min_frame_idx, max_frame_idx + 1):
        print("Frame %05d/%05d" % (frame_idx, max_frame_idx))
        mask = frame_indices == frame_idx
        rows = detections_in[mask]

        if frame_idx not in frame_indices:
            print("WARNING could not find image for frame %d" % frame_idx)
            continue
        camera.set(cv2.CAP_PROP_POS_FRAMES, frame_idx-1);
        (grabbed, bgr_image) = camera.read()
        # bgr_image = cv2.imread(image_filenames[frame_idx], cv2.IMREAD_COLOR)
        rows[:, 4:6] -= rows[:, 2:4]  # convert to [x1,y1,x2,y2] to [x1,y1,w,h]
        features = encoder(bgr_image, rows[:, 2:6].copy())
        detections_out += [np.r_[(row, feature)] for row, feature
                           in zip(rows, features)]

    # output_filename = os.path.join(output_dir, "%s.npy" % sequence)
    np.save(name_out, np.asarray(detections_out), allow_pickle=False)

# Track objects

In [ ]:
    frame_idx =0;
    min_frame_idx = int(detections_file[:, 0].min())
    max_frame_idx = int(detections_file[:, 0].max())
    min_confidence = 0.95
    min_detection_height = 0
    nms_max_overlap = 0.3
    if (display):
        plt.ion()
        fig = plt.figure()

    while True:
        (grabbed, frame) = camera.read()
        frame_idx += 1;
        print("Processing frame %05d" % frame_idx)

        # Load image and generate detections.
        detections = create_detections(detections_file, frame_idx-1, min_detection_height)
        detections = [d for d in detections if d.confidence >= min_confidence]

        # Run non-maxima suppression.
        boxes = np.array([d.tlwh for d in detections])
        scores = np.array([d.confidence for d in detections])
        indices = preprocessing.non_max_suppression(boxes, nms_max_overlap, scores)
        detections = [detections[i] for i in indices]

        # Update tracker.
        tracker.predict()
        tracker.update(detections)

        # Update visualization.
        if display:
            ax1 = fig.add_subplot(111, aspect='equal')
            # fn = 'mot_benchmark/%s/%s/img1/%06d.jpg'%(phase,seq,frame)
            ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.title(' Tracked Targets')

        # Store results.
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlwh()
            results.append([
                frame_idx, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3]])

         
            if (display):
                ax1.add_patch(patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], fill=False, lw=3,ec=colours[track.track_id % 32, :]))
                ax1.set_adjustable('box-forced')
                plt.text(bbox[0], bbox[1], str(track.track_id))

        if(display):
          fig.canvas.flush_events()
          plt.draw()
          ax1.cla()

        if grabbed == False:
            break